# <center> Garmin Watch Data 

In [1]:
import json
import pandas as pd
import numpy as np
import os
from datetime import date, timedelta, datetime
# os.chdir('/Users/jayrajparmar/Documents/side_project/health_data_tracking')

import sys

sys.path.insert(1,  os.path.abspath('..'))

import basic_methods as bm
import time
# import pathlib

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# f = open('/Users/jayrajparmar/Documents/side_project/health_data_tracking/Garmin/jayrajparmar009@gmail.com_0_summarizedActivities.json', "r")



f = open(os.path.join(os.path.dirname('garmin_summarized.json'), 
                      'garmin_summarized.json'))

data = json.loads(f.read())

json_data = data[0]['summarizedActivitiesExport']

lis = []

for i in range(len(json_data)):
    df = pd.DataFrame([json_data[i]], columns = json_data[i].keys())
    lis.append(df)
df_garmin_exec = pd.concat(lis)


In [3]:
df_garmin_exec.head()

,activityId,uuidMsb,uuidLsb,name,activityType,userProfileId,timeZoneId,beginTimestamp,eventTypeId,rule,sportType,startTimeGmt,startTimeLocal,duration,distance,avgSpeed,avgHr,maxHr,maxRunCadence,steps,calories,bmrCalories,avgStrideLength,avgFractionalCadence,maxFractionalCadence,elapsedDuration,movingDuration,deviceId,avgDoubleCadence,maxDoubleCadence,summarizedDiveInfo,manufacturer,lapCount,waterEstimated,purposeful,autoCalcCalories,favorite,pr,elevationCorrected,decoDive,atpActivity,parent,elevationGain,elevationLoss,minElevation,maxElevation,maxVerticalSpeed,maxSpeed,startLongitude,startLatitude,vO2MaxValue,locationName,endLongitude,endLatitude,avgVerticalSpeed,floorsClimbed,floorsDescended,summarizedExerciseSets,activeSets,totalSets,totalReps,minRespirationRate,maxRespirationRate,avgRespirationRate,startStress,endStress,differenceStress
0,8959734435,-5634344091463760210,-5357954891567381561,Elliptical,elliptical,82576128,149,1654442920000,9,private,FITNESS_EQUIPMENT,"1,654,442,920,000.0000","1,654,428,520,000.0000","2,983,868.8965",0.0000,0.0000,148.0000,164.0000,79.0000,"2,258.0000","2,832.4535",318.4415,0.0000,0.1875,0.5000,"2,983,868.8965",0.0000,3317189130,48.3750,159.0000,{},GARMIN,1,298.0000,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8952433395,-1097609307350545764,-5078813204589093572,Cardio,indoor_cardio,82576128,149,1654352911000,9,private,FITNESS_EQUIPMENT,"1,654,352,911,000.0000","1,654,338,511,000.0000","144,330.9937",0.0000,0.0000,84.0000,89.0000,NaN,NaN,37.7102,12.5701,NaN,0.0000,0.0000,"144,330.9937",0.0000,3317189130,NaN,NaN,{},GARMIN,1,14.0000,False,False,False,False,False,False,False,False,0.0000,0.0000,"-17,939.9994","-17,700.0000",0.0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8917024003,291049867221549316,-6719738246007706917,Cardio,indoor_cardio,82576128,149,1653836425000,9,private,TRAINING,"1,653,836,425,000.0000","1,653,822,025,000.0000","6,977,541.0156",0.0000,0.0000,121.0000,151.0000,NaN,NaN,"3,456.7665",745.8236,NaN,0.0000,0.0000,"6,977,541.0156",0.0000,3317189130,NaN,NaN,{},GARMIN,1,697.0000,False,False,False,False,False,False,False,False,0.0000,0.0000,"-9,700.0000","-8,459.9998",0.1333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8899731934,2752421229750862679,-6627922957386941904,Cardio,indoor_cardio,82576128,149,1653606038000,9,private,TRAINING,"1,653,606,038,000.0000","1,653,591,638,000.0000","3,719,052.9785",0.0000,0.0000,153.0000,188.0000,NaN,NaN,"3,230.5054",398.0519,NaN,0.0000,0.0000,"3,719,052.9785",0.0000,3317189130,NaN,NaN,{},GARMIN,1,371.0000,False,False,False,False,False,False,False,False,0.0000,0.0000,"-3,179.9999","-3,179.9999",0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8896332721,-8383978575552557445,-4936977240004103301,Cardio,indoor_cardio,82576128,149,1653563124000,9,private,FITNESS_EQUIPMENT,"1,653,563,124,000.0000","1,653,548,724,000.0000","6,806,884.7656",0.0000,0.0000,136.0000,173.0000,NaN,NaN,"4,156.4998",729.0635,NaN,0.0000,0.0000,"6,806,884.7656",0.0000,3317189130,NaN,NaN,{},GARMIN,1,680.0000,False,False,False,False,False,False,False,False,0.0000,0.0000,"-2,100.0000","-1,400.0000",0.0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def duration_measure(col):
#     mili_sec = int(col)
    sec = int((col/1000))
    minutes = int((sec)/60)
    return minutes


def date_range_generator(start_dt, end_dt):
    """
    Function for generating date range between start and end date;
    It has 2 arguments to be added:
        * start_dt = datetime value for the start date
        * end_dt = datetime value for the end date
    """
    lis = []
    while start_dt < end_dt:
        start_dt += timedelta(days=1)
        lis.append(start_dt)
    df = pd.DataFrame(lis, columns=['date_range'])
    df['date_range'] = pd.to_datetime(df['date_range'], errors='coerce')
    return df

In [8]:
df_garmin_imp_cols = df_garmin_exec[['activityId','name','activityType','startTimeLocal','duration',
                                       'avgHr','maxHr','calories','bmrCalories','waterEstimated']]

df_garmin_imp_cols['datetime'] = pd.to_datetime(df_garmin_imp_cols['startTimeLocal'].map(lambda x: time.strftime(
                                                        '%Y-%m-%d %H:%M:%S', time.localtime(x/1000))))

df_garmin_imp_cols['date'] = df_garmin_imp_cols['datetime'].dt.date

df_garmin_imp_cols['duration'] = df_garmin_imp_cols['duration'].astype(int)

df_garmin_imp_cols['duration_minutes'] = df_garmin_imp_cols['duration'].map(lambda x: duration_measure(x))

df_garmin_imp_cols.columns = ['activityId', 'name', 'activityType', 'startTimeLocal', 'duration',
       'avgHr', 'maxHr', 'calories', 'bmrCalories', 'waterEstimated',
       'datetime', 'date', 'duration_minutes']

In [9]:
df_garmin_imp_cols = df_garmin_imp_cols[['datetime','date','duration_minutes', 'activityType','avgHr','maxHr',
                                         'bmrCalories','calories','waterEstimated']]

df_garmin_imp_cols['date'] = pd.to_datetime(df_garmin_imp_cols['date'], errors='coerce')

In [10]:
d = {
    'duration_minutes': np.sum,
    'avgHr': np.mean,
    'maxHr': np.max,
    'bmrCalories': np.sum,
    'calories': np.sum,
    'waterEstimated': np.sum
}

df_garmin_imp_cols = df_garmin_imp_cols.groupby('date').agg(d).reset_index()

In [11]:
date_range = date_range_generator((min(df_garmin_imp_cols.date.dt.date)),(max(df_garmin_imp_cols.date.dt.date)))
final_df1 = date_range.merge(df_garmin_imp_cols, how='left', left_on='date_range', right_on='date')

In [11]:
df_garmin_imp_cols.tail()

,datetime,date,duration_minutes,activityType,avgHr,maxHr,bmrCalories,calories,waterEstimated
0,2020-01-06 02:26:25,2020-01-06,45,indoor_cardio,138.0000,169.0000,NaN,"2,019.5896",272.0000
0,2020-01-06 02:02:42,2020-01-06,23,strength_training,128.0000,162.0000,NaN,925.9944,140.0000
0,2020-01-06 01:55:23,2020-01-06,5,strength_training,123.0000,145.0000,NaN,213.6910,31.0000
0,2020-01-05 04:32:32,2020-01-05,86,indoor_cardio,155.0000,182.0000,NaN,"4,663.4923",516.0000
0,2020-01-04 05:46:45,2020-01-04,53,indoor_cardio,154.0000,190.0000,NaN,"3,046.1445",320.0000


In [13]:
final_df1

,date_range,date,duration_minutes,avgHr,maxHr,bmrCalories,calories,waterEstimated
0,2020-01-05,2020-01-05,86.0000,155.0000,182.0000,0.0000,"4,663.4923",516.0000
1,2020-01-06,2020-01-06,73.0000,129.6667,169.0000,0.0000,"3,159.2751",443.0000
2,2020-01-07,2020-01-07,105.0000,133.6667,164.0000,0.0000,"4,085.2695",636.0000
3,2020-01-08,2020-01-08,87.0000,112.3333,171.0000,0.0000,"2,966.5342",530.0000
4,2020-01-09,2020-01-09,107.0000,141.5000,174.0000,0.0000,"4,638.3521",648.0000
5,2020-01-10,2020-01-10,74.0000,132.0000,168.0000,0.0000,"2,933.0140",446.0000
6,2020-01-11,2020-01-11,56.0000,154.0000,183.0000,0.0000,"3,293.3557",340.0000
7,2020-01-12,2020-01-12,93.0000,157.0000,179.0000,0.0000,"5,275.2352",563.0000
8,2020-01-13,2020-01-13,125.0000,132.5000,174.0000,0.0000,"5,204.0048",752.0000
9,2020-01-14,2020-01-14,72.0000,131.0000,166.0000,0.0000,"3,058.7146",433.0000


In [4]:
import garmin as gm



In [5]:
final_df = gm.final_df_prep()

/Users/jayrajparmar/Documents/side_project/health_data_tracking/Garmin


In [3]:
final_df

,date_range,date,duration_minutes,avgHr,maxHr,bmrCalories,calories,waterEstimated
0,2019-01-02,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-03,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-04,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-05,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-06,NaT,NaN,NaN,NaN,NaN,NaN,NaN
5,2019-01-07,NaT,NaN,NaN,NaN,NaN,NaN,NaN
6,2019-01-08,NaT,NaN,NaN,NaN,NaN,NaN,NaN
7,2019-01-09,NaT,NaN,NaN,NaN,NaN,NaN,NaN
8,2019-01-10,NaT,NaN,NaN,NaN,NaN,NaN,NaN
9,2019-01-11,NaT,NaN,NaN,NaN,NaN,NaN,NaN
